<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/114_1_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. 安裝所有必要的套件
# 使用 %%capture 來抑制安裝過程中的輸出訊息
try:
    import gradio as gr
    import gspread
    import pandas as pd
    import google.generativeai as genai
    import selenium # <-- 新增 selenium 檢查
    print("所有套件皆已安裝。")
except ImportError:
    print("正在安裝必要的套件...")
    # 新增 selenium 套件
    %pip install gradio gspread google-auth-oauthlib google-auth-httplib2 pandas scikit-learn beautifulsoup4 google-generativeai lxml selenium
    print("套件安裝完成！")

正在安裝必要的套件...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 37.4 MB/s eta 0:00:00
套件安裝完成！


In [2]:
# @title 2. 匯入套件與進行 Google 服務授權
import os
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import google.generativeai as genai
import gspread
from google.colab import auth
from google.auth import default
import gradio as gr
from google.colab import userdata

In [3]:
# --- Selenium 相關匯入與設定 ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def setup_selenium_driver():
    """在 Colab 環境中設定 Selenium WebDriver"""
    # 設定 Chrome 選項
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 無頭模式，不在 Colab 中顯示瀏覽器 UI
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

    # Colab 中不需要 service，可以直接初始化
    driver = webdriver.Chrome(options=chrome_options)
    return driver


print("正在進行 Google 服務授權...")
# 授權 Colab 存取 Google Drive/Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("Google 服務授權成功！")

正在進行 Google 服務授權...
Google 服務授權成功！


In [4]:
# @title 3. 設定區：Google Sheet 與 Gemini API
# --- Google Sheet 設定 ---
# 請確認您已將 Colab 存取權限的服務帳戶 email 加入到此 Google Sheet 的編輯權限中
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1FODK44udgJJuvZLj9BEER6n7iMaTQyxRHRFL1KO1yBQ/edit?usp=sharing"
RAW_DATA_SHEET_NAME = "爬蟲原始資料"
STATS_SHEET_NAME = "統計結果"

# --- Gemini API 設定 ---
import google.generativeai as genai # 將匯入移到這裡確保 genai 已定義
from google.colab import userdata
try:
    # 從 Colab Secrets 讀取 API Key
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("Gemini API 金鑰設定成功。")
except Exception as e:
    print(f"無法讀取 Gemini API 金鑰，請確認您已在 Colab 的密鑰 (Secrets) 中設定 'GOOGLE_API_KEY'。錯誤訊息: {e}")
    GEMINI_API_KEY = None

Gemini API 金鑰設定成功。


In [5]:
# --- 其他設定 ---
TARGET_URL = "https://www.chiefdelphi.com/"
TOP_N_KEYWORDS = 15 # 設定要輸出的熱詞數量

In [6]:
# @title 4. 定義核心功能函式

import gradio as gr # 將匯入移到這裡確保 gr 已定義
import pandas as pd # 確保 pandas 在這裡可用
from sklearn.feature_extraction.text import TfidfVectorizer # 確保 TfidfVectorizer 在這裡可用
from bs4 import BeautifulSoup # 確保 BeautifulSoup 在這裡可用
import requests # 確保 requests 在這裡可用
import time # 確保 time 在這裡可用


# --- Selenium 相關匯入與設定 ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def setup_selenium_driver():
    """在 Colab 環境中設定 Selenium WebDriver"""
    # 設定 Chrome 選項
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 無頭模式，不在 Colab 中顯示瀏覽器 UI
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

    # Colab 中不需要 service，可以直接初始化
    driver = webdriver.Chrome(options=chrome_options)
    return driver

# --- 其他設定 ---
TARGET_URL = "https://www.chiefdelphi.com/"
TOP_N_KEYWORDS = 15 # 設定要輸出的熱詞數量
INSIGHTS_SHEET_NAME = "AI分析結果" # 新增一個工作表名稱用於存放 AI 分析結果

def scrape_chief_delphi():
    """使用 Selenium 爬取 Chief Delphi 首頁的最新主題標題"""
    print("STEP 1: 開始使用 Selenium 爬取 Chief Delphi 論壇...")
    driver = None
    try:
        driver = setup_selenium_driver()
        driver.get(TARGET_URL)

        # 等待目標元素出現（最多20秒），確保 JS 已載入完畢
        print("正在等待頁面元素載入...")
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.raw-link.raw-topic-link')))
        print("頁面元素載入完成。")

        soup = BeautifulSoup(driver.page_source, 'lxml')
        topic_links = soup.select('a.raw-link.raw-topic-link')
        titles = [link.get_text(strip=True) for link in topic_links]

        if not titles:
            print("警告：沒有爬取到任何主題標題，可能是網站結構已變更。")
            return []

        print(f"爬取成功！共找到 {len(titles)} 個主題。")
        return titles
    except Exception as e:
        print(f"使用 Selenium 爬取失敗，錯誤訊息：{e}")
        return None
    finally:
        if driver:
            driver.quit()
            print("Selenium WebDriver 已關閉。")

def write_to_google_sheet(spreadsheet_url, sheet_name, data):
    """將資料寫入指定的 Google Sheet 工作表"""
    print(f"STEP: 開始寫入資料至 Google Sheet '{sheet_name}'...")
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)
        try:
            worksheet = spreadsheet.worksheet(sheet_name)
            worksheet.clear()
        except gspread.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")

        if isinstance(data, list):
            if not data: # Handle empty list case
                df = pd.DataFrame()
            elif isinstance(data[0], dict): # Handle list of dictionaries
                df = pd.DataFrame(data)
            elif isinstance(data[0], list): # Handle list of lists
                 df = pd.DataFrame(data)
            else: # Assume list of simple values
                df = pd.DataFrame(data, columns=['Value'])
        elif isinstance(data, pd.DataFrame):
            df = data
        elif isinstance(data, str): # Handle single string
            df = pd.DataFrame([data], columns=['Content'])
        elif isinstance(data, tuple) and len(data) == 2 and all(isinstance(item, str) for item in data): # Handle tuple of two strings
             df = pd.DataFrame({'洞察摘要': [data[0]], '趨勢結論': [data[1]]})
        else:
            raise ValueError("不支援的資料格式")

        if not df.empty:
             worksheet.update([df.columns.values.tolist()] + df.values.tolist())
        else:
             worksheet.update([["No data to write"]])


        print("資料寫入 Google Sheet 成功！")
        return True
    except Exception as e:
        print(f"寫入 Google Sheet 失敗，錯誤訊息：{e}")
        return False

def analyze_text_from_sheet(spreadsheet_url, sheet_name, top_n=10):
    """從 Google Sheet 讀取資料並進行 TF-IDF 分析"""
    print("STEP: 從 Google Sheet 讀取資料並進行 TF-IDF 分析...")
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)
        worksheet = spreadsheet.worksheet(sheet_name)
        data = worksheet.get_all_values()

        if len(data) < 2:
            print("工作表中沒有足夠的資料可供分析。")
            return None, None

        df = pd.DataFrame(data[1:], columns=data[0])
        documents = df['Topic Title'].dropna().tolist()

        vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
        tfidf_matrix = vectorizer.fit_transform(documents)
        feature_names = vectorizer.get_feature_names_out()
        total_tfidf_scores = tfidf_matrix.sum(axis=0).tolist()[0]

        keywords_df = pd.DataFrame({'Keyword': feature_names, 'TF-IDF Score': total_tfidf_scores})
        top_keywords_df = keywords_df.sort_values(by='TF-IDF Score', ascending=False).head(top_n)
        top_keywords_df['TF-IDF Score'] = top_keywords_df['TF-IDF Score'].round(4)

        print(f"TF-IDF 分析完成！已找出前 {top_n} 個熱門關鍵字。")
        return top_keywords_df, documents
    except Exception as e:
        print(f"文字分析失敗，錯誤訊息：{e}")
        return None, None

def generate_insights_with_gemini(keywords, documents):
    """【已修改】使用 requests 直接呼叫 Gemini REST API 生成洞察摘要與結論"""
    if not GEMINI_API_KEY:
        return "Gemini API 金鑰未設定，無法生成摘要。", "請在 Colab 的密鑰(Secrets)功能中設定您的 API 金鑰。"

    print("STEP: 開始使用 requests 直接呼叫 Gemini REST API...")

    # 使用 gemini-flash-latest 模型
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-latest:generateContent?key={GEMINI_API_KEY}"

    keywords_str = ", ".join(keywords)
    sample_titles = "\n- ".join(pd.Series(documents).sample(min(5, len(documents))).tolist())

    prompt = f"""
    您是一位專業的 FRC (FIRST Robotics Competition) 領域的數據分析師。
    請根據以下從 Chief Delphi 論壇分析出的熱門關鍵字，以及一些原始的討論標題範例，生成一份簡潔的洞察報告。

    **分析資料:**
    - **熱門關鍵字:** {keywords_str}
    - **原始標題範例:**
    - {sample_titles}

    **您的任務:**
    請嚴格遵循以下格式輸出，不要有任何額外的說明文字：

    ---洞察摘要---
    1. [第一點洞察，一句話總結]
    2. [第二點洞察，一句話總結]
    3. [第三點洞察，一句話總結]
    4. [第四點洞察，一句話總結]
    5. [第五點洞察，一句話總結]

    ---趨勢結論---
    [基於以上洞察，撰寫一段約 120 字的綜合性結論，分析目前社群的討論焦點與潛在趨勢。]
    """

    payload = {
        "contents": [{
            "parts": [{
                "text": prompt
            }]
        }]
    }

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # 如果請求失敗 (例如 4xx 或 5xx)，則會引發 HTTPError

        result = response.json()
        # 檢查 'candidates' 和 'content' 是否存在
        if 'candidates' in result and len(result['candidates']) > 0 and 'content' in result['candidates'][0] and 'parts' in result['candidates'][0]['content'] and len(result['candidates'][0]['content']['parts']) > 0 and 'text' in result['candidates'][0]['content']['parts'][0]:
            generated_text = result['candidates'][0]['content']['parts'][0]['text']
        else:
            print("Gemini API 回應格式不正確或沒有生成內容。")
            print(f"完整回應: {result}")
            return "Gemini API 回應無效。", "無法從 API 回應中解析摘要和結論。"


        parts = generated_text.split("---趨勢結論---")
        summary = parts[0].replace("---洞察摘要---", "").strip()
        conclusion = parts[1].strip() if len(parts) > 1 else "無法生成結論。"

        print("Gemini REST API 摘要生成成功！")
        return summary, conclusion

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP 錯誤發生：{http_err}")
        print(f"伺服器回應：{response.text}")
        return f"Gemini API 呼叫失敗: {http_err}", "請檢查您的 API 金鑰是否有效且已啟用 Generative Language API。"
    except Exception as e:
        print(f"呼叫 Gemini API 時發生未知錯誤，錯誤訊息：{e}")
        return f"Gemini API 呼叫失敗: {e}", "請檢查您的網路連線或稍後再試。"




In [7]:
# @title 5. 建立主執行流程與 Gradio 介面

def run_full_pipeline(progress=gr.Progress()):
    """執行完整的自動化流程"""
    # 1. 爬蟲
    progress(0.1, desc="STEP 1/6: 正在爬取 Chief Delphi 論壇資料...")
    titles = scrape_chief_delphi()
    if titles is None or not titles:
        return "爬蟲失敗或未找到資料，流程中止。", "", "", ""
    time.sleep(1)

    # 2. 寫入原始資料
    progress(0.25, desc="STEP 2/6: 正在將原始資料寫入 Google Sheet...")

    # --- 【修改點】 ---
    # 將 list 轉換為有正確欄位名稱 "Topic Title" 的 DataFrame
    titles_df = pd.DataFrame(titles, columns=['Topic Title'])
    # 將修改後的 DataFrame 寫入 Sheet
    write_to_google_sheet(SPREADSHEET_URL, RAW_DATA_SHEET_NAME, titles_df)
    # --- 【修改結束】 ---

    time.sleep(1)

    # 3. 分析
    progress(0.4, desc="STEP 3/6: 正在進行 TF-IDF 關鍵字分析...")
    top_keywords_df, documents = analyze_text_from_sheet(SPREADSHEET_URL, RAW_DATA_SHEET_NAME, top_n=TOP_N_KEYWORDS)
    if top_keywords_df is None:
        return "文字分析失敗，流程中止。", "", "", ""
    time.sleep(1)

    # 4. 回寫統計結果
    progress(0.55, desc="STEP 4/6: 正在將統計結果回寫至 Google Sheet...")
    write_to_google_sheet(SPREADSHEET_URL, STATS_SHEET_NAME, top_keywords_df)
    time.sleep(1)

    # 5. AI 生成摘要
    progress(0.7, desc="STEP 5/6: 正在呼叫 Gemini API 生成洞察報告...")
    keywords_list = top_keywords_df['Keyword'].tolist()
    summary, conclusion = generate_insights_with_gemini(keywords_list, documents)
    time.sleep(1)

    # 6. 寫入 AI 分析結果
    progress(0.9, desc="STEP 6/6: 正在將 AI 分析結果寫入 Google Sheet...")
    write_to_google_sheet(SPREADSHEET_URL, INSIGHTS_SHEET_NAME, (summary, conclusion))

    status_message = f"流程執行完畢！\n\n- 成功爬取 {len(titles)} 筆資料。\n- 找出了 {len(keywords_list)} 個熱門關鍵字。\n- 已將所有結果更新至您的 Google Sheet 的 '{RAW_DATA_SHEET_NAME}', '{STATS_SHEET_NAME}' 和 '{INSIGHTS_SHEET_NAME}' 工作表。"
    keywords_output = top_keywords_df.to_string(index=False)

    return status_message, keywords_output, summary, conclusion

# --- 建立 Gradio 介面 ---
# (此處介面程式碼無需變更)
print("\n" + "="*50)
print("Gradio 介面即將啟動...")
print("請點擊下方產生的公開連結 (Public URL) 來開啟操作介面。")
print("="*50 + "\n")

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(f"""
        # Chief Delphi 論壇趨勢分析工具
        **這是一個自動化流程，整合了網頁爬蟲、數據分析與 AI 摘要生成。**

        **操作流程:**
        1. 點擊下方的 **「🚀 開始完整分析流程」** 按鈕。
        2. 系統將自動執行所有步驟（爬蟲 → 寫入原始資料 → 分析 → 回寫統計結果 → AI 生成摘要 → 寫入 AI 分析結果）。
        3. 等待約 1-2 分鐘，結果將會顯示在下方的輸出框中。
        4. 您也可以隨時前往您的 [Google Sheet]({SPREADSHEET_URL}) 查看即時更新的數據。
        """
    )

    with gr.Row():
        run_button = gr.Button("🚀 開始完整分析流程", variant="primary")

    with gr.Accordion("📊 執行狀態與結果", open=True):
        status_output = gr.Textbox(label="系統執行狀態", lines=5, interactive=False)
        keywords_output = gr.Textbox(label=f"TOP {TOP_N_KEYWORDS} 熱門關鍵字 (TF-IDF)", lines=8, interactive=False)
        summary_output = gr.Textbox(label="🤖 Gemini AI - 洞察摘要", lines=7, interactive=False)
        conclusion_output = gr.Textbox(label="🤖 Gemini AI - 趨勢結論", lines=6, interactive=False)

    run_button.click(
        fn=run_full_pipeline,
        inputs=[],
        outputs=[status_output, keywords_output, summary_output, conclusion_output]
    )

demo.launch(debug=True, share=True)


Gradio 介面即將啟動...
請點擊下方產生的公開連結 (Public URL) 來開啟操作介面。

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://60f3a72a34a0f9d5f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


STEP 1: 開始使用 Selenium 爬取 Chief Delphi 論壇...
正在等待頁面元素載入...
頁面元素載入完成。
爬取成功！共找到 30 個主題。
Selenium WebDriver 已關閉。
STEP: 開始寫入資料至 Google Sheet '爬蟲原始資料'...
資料寫入 Google Sheet 成功！
STEP: 從 Google Sheet 讀取資料並進行 TF-IDF 分析...
TF-IDF 分析完成！已找出前 15 個熱門關鍵字。
STEP: 開始寫入資料至 Google Sheet '統計結果'...
資料寫入 Google Sheet 成功！
STEP: 開始使用 requests 直接呼叫 Gemini REST API...
Gemini REST API 摘要生成成功！
STEP: 開始寫入資料至 Google Sheet 'AI分析結果'...
資料寫入 Google Sheet 成功！
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://60f3a72a34a0f9d5f4.gradio.live
